In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re
import string
import random
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import mean_squared_error

In [ ]:
#@title Record Each Cell's Execution Time
!pip install ipython-autotime

%load_ext autotime

Loading data

In [ ]:
data = pd.read_csv('dataset.csv',encoding="utf-8") # utf-8 because of the german text

# Explore the Dataset

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
def kldb_devision(_data, _start_digit,_end_digit):
  _data['col'] = _data['kldb'].astype(str)
  _data['kldb_new'] = _data['col'].str[_start_digit:_end_digit]


In [ ]:
# data['ctotal'] = data.cberuf.astype(str).str.cat(data.chelp1.astype(str), sep=', ')
# data['ctotal1'] = data.ctotal.astype(str).str.cat(data.chelp2.astype(str), sep=', ')
# data['ctotal2'] = data.ctotal1.astype(str).str.cat(data.chelp3.astype(str), sep=', ')
# data['ctotal3'] = data.ctotal2.astype(str).str.cat(data.chelp4.astype(str), sep=', ')
# # #data['ctotal4'] = data.ctotal3.astype(str).str.cat(data.chelp5.astype(str), sep=', ')
# # # df['cberuf_cfunktion'] = df.cberuf_cfunktion_cfunktion.astype(str).str.cat(df.cfunkti2.astype(str), sep=', ')
# # # df.cberuf_cfunktion[0]
# data.cberuf = data.ctotal3
# data = data.drop(columns="ctotal")
# data = data.drop(columns="ctotal1")
# data = data.drop(columns="ctotal2")
# data = data.drop(columns="ctotal3")

time: 610 µs (started: 2022-09-02 08:14:44 +00:00)


In [ ]:
#data.columns=['id'	,'text'	,'category','p_lfdnr',	'col','col1'	,'chelp5','chelp1',	'chelp2',	'chelp3',	'chelp4','']
data.columns=['id','id'	,'text'	,'category','p_lfdnr',	'col','col1'	,'chelp5','chelp1',	'chelp2',	'chelp3',	'chelp4','ctotal',	'ctotal1',	'ctotal2',	'ctotal3',	'ctotal4']

time: 1.28 ms (started: 2022-09-02 08:14:44 +00:00)


 Data Analysis
  * The number of reviews of each category:

In [ ]:
data.category.value_counts()

 Adding Category Id

In [ ]:
data["category"] = data["category"].astype('category') #By converting an existing Series or column to a category dtype
data.dtypes

In [ ]:
data["category_id"] = data["category"].cat.codes
data.tail()

In [ ]:
data.dtypes

##Dictionaries for future use

In [ ]:
id_to_category = pd.Series(data.category.values,index=data.category_id).to_dict()
id_to_category

In [ ]:
category_to_id= {v:k for k,v in id_to_category.items()}
category_to_id

In [ ]:
number_of_categories = len(category_to_id)
number_of_categories

 Some necessary variables

In [ ]:
#limit the number of samples to be used in code runs
data_size= 427230

time: 619 µs (started: 2022-09-02 08:15:16 +00:00)


In [ ]:
vocab_size = 90000  # Only consider the top 90k words
maxlen = 260  # Max sequence size

time: 479 µs (started: 2022-09-02 08:15:16 +00:00)


In [ ]:
# save features and targets from the 'data' (raw data)
features, targets = data['text'], data['category_id']

time: 7.67 ms (started: 2022-09-02 08:15:16 +00:00)


Spliting the data into train and test sets

* We should split the data before the preprocessing. 

* In order to avoid data leaks, some preprocesses should not be applied on the test set.

In [ ]:
train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        train_size=0.9,
        test_size=0.1,
        # random but same for all run, also accurancy depends on the
        # selection of data e.g. if we put 10 then accuracy will be 1.0
        # in this example
        random_state=46,
        # keep same proportion of 'target' in test and target data
        #stratify=targets
    )

 Model

In [ ]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__()
        self.embed_dim=embed_dim
        self.num_heads = num_heads
        self.ff_dim =ff_dim
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        # for masked-self attention add the mask:
        # causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        # attention_output = self.att(inputs, inputs,attention_mask=causal_mask)
        
        attention_output = self.att(inputs, inputs) 
        
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    # https://newbedev.com/saving-keras-models-with-custom-layers
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'att': self.att,
            'ffn': self.ffn,
            #'layernorm1': self.layernorm1,
            #'layernorm2':self.layernorm2,
            'dropout1':self.dropout1,
            'dropout2':self.dropout2,
            'embed_dim': self.embed_dim,
            'num_heads':self.num_heads,
            'ff_dim':self.ff_dim 

        })
        return config
        #tf.keras.models.save_model(model, 'model.h5')
        #new_model = tf.keras.models.load_model('model.h5', custom_objects={'CustomLayer': CustomLayer})


In [ ]:
class TokenPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size,  embed_dim, **kwargs):
        super(TokenPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, X):
        maxlen = tf.shape(X)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        X = self.token_emb(X)
        return X + positions 
        
    # https://newbedev.com/saving-keras-models-with-custom-layers
    def get_config(self):
        config = super(TokenPositionEmbedding, self).get_config()
        config.update({
            'token_emb': self.token_emb,
            'pos_emb': self.pos_emb,
            'maxlen': self.maxlen,
            'vocab_size': self.vocab_size,
            'embed_dim': self.embed_dim
        })
        return config
        #tf.keras.models.save_model(model, 'model.h5')
        #new_model = tf.keras.models.load_model('model.h5', custom_objects={'CustomLayer': CustomLayer})
        

In [ ]:
embed_dim = 256  # Embedding size for each token
num_heads = 2  # Number of attention heads
feed_forward_dim = 256  # Hidden layer size in feed forward network inside transformer


def create_model():
    inputs_tokens = layers.Input(shape=(maxlen,), dtype=tf.int32)
    embedding_layer = TokenPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs_tokens)
    transformer_block1 = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    transformer_block2 = TransformerBlock(embed_dim, num_heads, feed_forward_dim)
    x = transformer_block1(x)
    x = transformer_block2(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(number_of_categories)(x)
    model = keras.Model(inputs=inputs_tokens, outputs=outputs)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_fn  = tf.keras.metrics.SparseCategoricalAccuracy()
    model.compile(optimizer="adam", loss=loss_fn, metrics=metric_fn)  
    
    return model
my_model=create_model()

In [ ]:
my_model.summary()

In [ ]:
tf.keras.utils.plot_model(my_model,show_shapes=True)

In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/models
%pwd

In [ ]:
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False)

In [ ]:
X,y = train_features,train_targets

balanced_accuracy_scores = []
matthews_corrcoef_scores = []
f1_scores = []
conf_matrix_list_of_arrays = []

# prepare cross validation
n=5
seed=1
k_fold = KFold(n_splits=n, random_state=seed, shuffle=True)

for train_index, test_index in k_fold.split(X,y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  X_train_dtm = X_train
  X_test_dtm = X_test
  history = my_model.fit(X_train_dtm, y_train, verbose=1, epochs=1, callbacks=[model_checkpoint_callback])
  y_pred_class = my_model.predict(X_test_dtm)

  y_pred_class = np.argmax(y_pred_class,axis=1)
  
  conf_matrix = confusion_matrix(y_test, y_pred_class)
  conf_matrix_list_of_arrays.append(conf_matrix)
  
  balanced_accuracy_scores.append(balanced_accuracy_score(y_test, y_pred_class))
  matthews_corrcoef_scores.append(matthews_corrcoef(y_test, y_pred_class))
  f1_scores.append(f1_score(y_test, y_pred_class, average='weighted'))

Average sparse_categorical_accuracy: 0.9643

In [ ]:
mean_of_conf_matrix_arrays = np.mean(conf_matrix_list_of_arrays, axis=0)
print('Mean of conf_matrix: ', mean_of_conf_matrix_arrays)

In [ ]:
conf_mat = mean_of_conf_matrix_arrays.astype(int)

In [ ]:
category_id_df = data[['category', 'category_id']].drop_duplicates()

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=category_id_df.category.values, 
            yticklabels=category_id_df.category.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title("CONFUSION MATRIX \n", size=16);

Metrics

In [ ]:
balanced_accuracy_scores = np.array(balanced_accuracy_scores)
print('Mean of balanced_accuracy_scores: ', np.mean(balanced_accuracy_scores, axis=0))

In [ ]:
matthews_corrcoef_scores = np.array(matthews_corrcoef_scores)
print('Mean of matthews_corrcoef_scores: ', np.mean(matthews_corrcoef_scores, axis=0))

In [ ]:
f1_scores = np.array(f1_scores)
print('Mean of f1_scores: ', np.mean(f1_scores, axis=0))

**Test the Model**

In [ ]:
test_features_vc = test_features
y_pred_test = my_model.predict(test_features_vc)
y_pred_test = np.argmax(y_pred_test,axis=1)
 
balanced_accuracy_score_test = balanced_accuracy_score(test_targets, y_pred_test)
print('balanced_accuracy_score_test :',balanced_accuracy_score_test)
matthews_corrcoef_score_test = matthews_corrcoef(test_targets, y_pred_test)
print('matthews_corrcoef_score_test :',matthews_corrcoef_score_test)
f1_score_test = f1_score(test_targets, y_pred_test, average='weighted')
print('f1_score_test :',f1_score_test)


In [ ]:
new_review = ""
predictions=my_model.predict([new_review])
for pred in predictions:
  print(id_to_category[np.argmax(pred)])

In [ ]:
test = pd.read_csv('TEST.csv')

In [ ]:
# test['ctotal'] = test.cberuf.astype(str).str.cat(test.chelp2.astype(str), sep=', ')
# # #data['ctotal1'] = data.ctotal.astype(str).str.cat(data.chelp2.astype(str), sep=', ')
# # # data['ctotal2'] = data.ctotal1.astype(str).str.cat(data.chelp3.astype(str), sep=', ')
# # # data['ctotal3'] = data.ctotal2.astype(str).str.cat(data.chelp4.astype(str), sep=', ')
# # # #data['ctotal4'] = data.ctotal3.astype(str).str.cat(data.chelp5.astype(str), sep=', ')
# # # # df['cberuf_cfunktion'] = df.cberuf_cfunktion_cfunktion.astype(str).str.cat(df.cfunkti2.astype(str), sep=', ')
# # # # df.cberuf_cfunktion[0]
# test.cberuf = test.ctotal
# test = test.drop(columns="ctotal")

In [ ]:
#test = pd.read_csv('TEST.csv')
jobs = test.cberuf
#last_kldb=test.predict


In [ ]:
# test['col'] = test['all-kldb'].astype(str)

# # make the new columns using string indexing
# test['kldb0'] = test['col'].str[4:5]
# #test.kldb=test.kldb0
# true = []
# true = test.kldb0
# all = []
# all=test.col

In [ ]:
#true=[]
true = test.kldb

time: 972 µs (started: 2022-09-02 08:30:49 +00:00)


In [ ]:

predictions=my_model.predict(jobs)
result = []
arr = []
#lastKldb=[]
#lastKldb =test.last_predict
i=0
for  i in range(len(predictions)):
  arr.append(id_to_category[np.argmax(predictions[i])])
  arr.append(jobs[i])
  arr.append(true[i])
  #arr.append(all[i])
  #arr.append(last_kldb[i])
  result.append(arr)
  arr=[]

In [ ]:
result
dp = pd.DataFrame(result)
dp.columns=['predict','occupation','true']
dp.to_csv('result.csv')
from google.colab import files
files.download("result.csv")


In [ ]:
dp['x'] = dp.last_kldb.astype(str).str.cat(dp.predict.astype(str), sep='')
dp.last_kldb=dp.x

save the model

In [ ]:
tf.keras.models.save_model(my_model, 'MultiClassTextClassifier')